In [1]:
### Testing the hybrid method
### Train vanilla using N walkers
### Train hybrid using N - M walkers, M points for mesh domain

### Also test whether neural network can approximate a given function

import torch
import numpy as np
import time
import math

import DRLPDE.create as create
import DRLPDE.neuralnets as neuralnets
import DRLPDE.train as train
import DRLPDE.parameters_solver as solver

import examples.hybrid as param

torch.set_default_dtype(torch.float32)

use_cuda=torch.cuda.is_available()
dev = torch.device("cuda:0" if use_cuda else "cpu")

In [2]:
hybrid_method = True

if hybrid_method:
    model_name = 'hybrid'
else:
    model_name = 'vanilla'

num_error = 2**15

In [3]:
# Training parameters

num_mesh = solver.num_mesh
solvemesh_every = solver.solvemesh_every
h = solver.h

mesh = param.mesh
num = solver.numpts
if hybrid_method:
    num = num - (num_mesh-2)**2
else:
    mesh.pop()


# Organize parameters
input_dim = [param.x_dim, param.t_dim, param.hyper_dim]

input_range = param.boundingbox + param.t_range + param.hyper_range

there_are_walls = any(param.list_of_walls)
there_are_solids = any(param.solid_walls)
there_are_inletoutlets = any(param.inlet_outlet)
there_are_meshes = any(mesh)
unsteady = bool(param.t_dim)

#num = solver.numpts
numbatch = solver.numbatch
resample_every = solver.resample_every
reweight_every = solver.reweight_every
trainingsteps = solver.trainingsteps
print_every = solver.print_every

MyNeuralNetwork = neuralnets.FeedForwardNN

var_nn = {'depth': solver.nn_depth,
        'width': solver.nn_width,
        'x_dim': input_dim[0],
        'is_unsteady': False,
        'output_dim': param.output_dim
        }

if param.loadmodel:
    model = MyNeuralNetwork(**var_nn).to(dev)
    model.load_state_dict(torch.load("savedmodels/" + param.loadmodel + ".pt"))
    print("Using model from savedmodels/" + param.loadmodel + ".pt")
else:
    model = MyNeuralNetwork(**var_nn).to(dev)

optimizer = torch.optim.Adam(model.parameters(), 
                                lr=solver.learningrate, 
                                betas=solver.adambeta, 
                                weight_decay=solver.weightdecay)


# Define the domain through its boundaries
Domain = create.SpaceDomain(param.boundingbox, 
                            param.list_of_walls, 
                            param.solid_walls, 
                            param.inlet_outlet, 
                            param.list_of_periodic_ends, 
                            mesh, 
                            param.init_con)

### Create Interior points and organize into batches
IntPoints = create.InteriorPoints(num, Domain, input_dim, input_range)
IntPoints_batch = torch.utils.data.DataLoader(IntPoints, batch_size=numbatch, shuffle=True)

### Create Dirichlet Wall points and organize into batches
if there_are_walls:
    num_wall = create.numBCpoints(num, input_dim, Domain, Domain.wall)
    BCPoints = create.BCPoints(num_wall, Domain, input_dim, input_range)

    BCPoints_batch = torch.utils.data.DataLoader(BCPoints, batch_size=numbatch, shuffle=True)

### Create Solid Wall points and organize into batches
if there_are_solids:
    num_solid = create.numBCpoints(num, input_dim, Domain, Domain.solid)
    SolidPoints = create.SolidWallPoints(num_solid, Domain, input_dim, input_range)
    SolidPoints_batch = torch.utils.data.DataLoader(SolidPoints, batch_size=numbatch, shuffle=True)

### Create Inlet/Outlet points and organize into batches
if there_are_inletoutlets:
    num_inout = create.numBCpoints(num, input_dim, Domain, Domain.inletoutlet)
    InletOutletPoints = create.InletOutletPoints(num_inout, Domain, input_dim, input_range)
    InletOutletPoints_batch = torch.utils.data.DataLoader(InletOutletPoints, batch_size=numbatch, shuffle=True)

if there_are_meshes:
    MeshPoints = create.MeshPoints(num_mesh, Domain.mesh[0], model, dev)
    MeshPoints_batch = torch.utils.data.DataLoader(MeshPoints, batch_size=numbatch, shuffle = True)

### Create Initial Condition points and organize into batches
if unsteady:
    num_ic = create.numICpoints(num, input_dim)
    ICPoints = create.ICPoints(num_ic, input_dim, input_range, Domain, param.init_con)
    ICPoints_batch = torch.utils.data.DataLoader(ICPoints, batch_size=numbatch, shuffle=True)

### Random points for calculating L2 error through Monte Carlo Integration
ErrorPoints = create.ErrorPoints(num_error, Domain, input_dim, input_range)
ErrorPoints_batch = torch.utils.data.DataLoader(ErrorPoints, batch_size=numbatch, shuffle = True)

### Method type
if solver.method == 'autodiff':
    import DRLPDE.autodiff as method
    var_train = {'diffusion': param.diffusion,
                    'forcing': param.forcing,
                    'x_dim': input_dim[0]
                }

elif solver.method == 'stochastic':
    import DRLPDE.stochastic as method
    var_train = {'diffusion': param.diffusion,
                'forcing': param.forcing, 
                'x_dim': input_dim[0],
                'domain': Domain,
                'dt': solver.dt,
                'num_ghost': solver.num_ghost, 
                'tol': solver.tol
                }

### PDE
if unsteady:
    if param.pde_type == 'NavierStokes':
        make_target = method.unsteadyNavierStokes
    elif param.pde_type == 'viscousBurgers':
        make_target = method.unsteadyViscousBurgers

else:
    if param.pde_type == 'NavierStokes':
        make_target = method.steadyNavierStokes
    elif param.pde_type == 'viscousBurgers':
        make_target = method.steadyViscousBurgers
    elif param.pde_type == 'Laplace':
        make_target = method.Laplace



In [6]:
### Train once

# Some parameters
do_resample = False
Linfloss_interior = 0.0
Linfloss_wall = 0.0
Linfloss_solid = 0.0
Linfloss_inletoutlet = 0.0
Linfloss_mesh = 0.0
Linfloss_ic = 0.0

weight_interior = 1e2
weight_wall = 1.0
weight_solid = 1.0
weight_inletoutlet = 1.0
weight_mesh = 1.0
weight_ic = 1.0

corners = torch.cartesian_prod( torch.tensor([-1.0, 1.0]), torch.tensor([-1.0, 1.0]))


### Train once ##
optimizer.zero_grad()

# Interior
L2loss_interior, Linfloss_interior, resample_interior = train.interior(IntPoints_batch, num, model, make_target, var_train, dev, weight_interior, Linfloss_interior, False)

# Boundary Values
if there_are_walls:
    L2loss_wall, Linfloss_wall, resample_wall = train.boundary(BCPoints_batch, num_wall, model, train.Dirichlet_target, dev, weight_wall, Linfloss_wall, False)

if there_are_solids:
    L2loss_solid, Linfloss_solid, resample_solid = train.boundary(SolidPoints_batch, num_solid, model, train.Dirichlet_target, dev, weight_solid, Linfloss_solid, False)

if there_are_inletoutlets:
    L2loss_inletoutlet, Linfloss_inletoutlet, resample_inletoutlet = train.boundary(InletOutletPoints_batch, num_inout, model, train.Inletoutlet_target, dev, weight_inletoutlet, Linfloss_inletoutlet, False)

if there_are_meshes:
    L2loss_mesh, Linfloss_mesh, [] = train.boundary(MeshPoints_batch, num_mesh, model, train.Dirichlet_target, dev, weight_mesh, Linfloss_mesh, False)

if unsteady:
    L2loss_ic, Linfloss_ic, resample_ic = train.boundary(ICPoints_batch, num_ic, model, train.Dirichlet_target, dev, weight_ic, Linfloss_ic, False)

L2error, Linferror = train.L2error(ErrorPoints_batch, num, model, param.bdry_con, dev)
Total_L2error = L2error.cpu().numpy()*np.ones(trainingsteps+1)
Total_Linferror = Linferror.cpu().numpy()*np.ones(trainingsteps+1)

optimizer.step()

print('No errors in first epoch, training will continue')    


No errors in first epoch, training will continue


In [8]:
###
### Continue training
###

start_time = time.time()

for step in range(trainingsteps):

    do_resample = False
    do_reweight = False
    #do_meshsolve = False

    #do_resample = (step+1) % resample_every == 0
    #do_reweight = (step+1) % reweight_every == 0
    do_meshsolve = (step+1) % solvemesh_every == 0
    
    optimizer.zero_grad()

    # Interior

    L2loss_interior, Linfloss_interior, resample_interior = train.interior(IntPoints_batch, num, model, make_target, var_train, dev, Linfloss_interior, do_resample)
    
    L2error, Linferror = train.L2error(ErrorPoints_batch, num, model, param.bdry_con, dev)
    Total_L2error[step+1] = L2error.cpu().numpy()
    Total_Linferror[step+1] = Linferror.cpu().numpy()

    # Walls
    if there_are_walls:
        if do_reweight:
            weight_wall = train.reweight(L2loss_interior, L2loss_wall)
        L2loss_wall, Linfloss_wall, resample_wall = train.boundary(BCPoints_batch, num_wall, model, train.Dirichlet_target, dev, weight_wall, Linfloss_wall, do_resample)
    
    if there_are_solids:
        if do_reweight:
            weight_solid = train.reweight(L2loss_interior, L2loss_solid)
        L2loss_solid, Linfloss_solid, resample_solid = train.boundary(SolidPoints_batch, num_solid, model, train.Dirichlet_target, dev, weight_solid, Linfloss_solid, do_resample)

    if there_are_inletoutlets:
        if do_reweight:
            weight_inletoutlet = train.reweight(L2loss_interior, L2loss_inletoutlet)
        L2loss_inletoutlet, Linfloss_inletoutlet, resample_inletoutlet = train.boundary(InletOutletPoints_batch, num_inout, model, train.Inletoutlet_target, dev, weight_inletoutlet, Linfloss_inletoutlet, do_resample)

    if there_are_meshes:
        L2loss_mesh, Linfloss_mesh, [] = train.boundary(MeshPoints_batch, num_mesh, model, train.Dirichlet_target, dev, weight_mesh, Linfloss_mesh, False)
        
        #L2loss_mesh_plus = (4.0*L2loss_mesh + 2.0*torch.sum( model(MeshPoints.Xwall)**2 ) + torch.sum( model(corners)**2 ))*(h**2)/4.0
        #L2error_mesh_plus = (4.0*train.L2error2(MeshPoints_batch, num_mesh, model, param.bdry_con) + 2.0*torch.sum( (model(MeshPoints.Xwall)) - param.bdry_con(MeshPoints.Xwall)**2 ) + torch.sum( (model(corners) - param.bdry_con(corners))**2) )*(h**2)/4.0 

        #Total_L2loss_hybrid[step+1] = ( L2loss_mesh_plus.detach().numpy() + L2loss_interior.data.numpy() )
        #Total_L2error_hybrid[step+1] =( L2error_mesh_plus.detach().numpy() + train.L2error(IntPoints_batch, num, model, param.bdry_con).numpy() )

    if unsteady:
        if do_reweight:
            weight_ic = train.reweight(L2loss_interior, L2loss_ic)
        L2loss_ic, Linfloss_ic, resample_ic = train.boundary(ICPoints_batch, num_ic, model, train.Dirichlet_target, dev, weight_ic, Linfloss_ic, do_resample)

    optimizer.step()

    # Move walkers (?)
    # TODO

    # Resample points
    if do_resample:
        if any(resample_interior):
            IntPoints.location[resample_interior,:] = create.generate_interior_points(torch.sum(resample_interior), input_dim, input_range, Domain)
            
        if there_are_walls:
            if any(resample_wall):
                BCPoints.location[resample_wall,:], BCPoints.value[resample_wall,:] = create.generate_boundary_points(torch.sum(resample_wall), input_dim, input_range, Domain.wall)
                
        if there_are_solids:
            if any(resample_solid):
                SolidPoints.location[resample_solid,:], SolidPoints.value[resample_solid,:] = create.generate_boundary_points(torch.sum(resample_solid), input_dim, input_range, Domain.solid)

        if there_are_inletoutlets:
            if any(resample_inletoutlet):
                InletOutletPoints.location[resample_inletoutlet,:], InletOutletPoints.value[resample_inletoutlet,:] = create.generate_boundary_points(torch.sum(resample_inletoutlet), input_dim, input_range, Domain.inletoutlet)

        if unsteady:
            if any(resample_ic):
                ICPoints.location[resample_ic,:] = create.generate_initial_points(torch.sum(resample_ic), input_dim, input_range, Domain)
                ICPoints.value[resample_ic,:] = param.init_con(ICPoints.location[resample_ic,:])

    if do_meshsolve:
        if there_are_meshes:
            b, dum = Domain.mesh[0].rhs(num_mesh, model)
            MeshPoints.value = MeshPoints.solveU(b)
        
            
    # Print statements
    if step == 0:
        current_time = time.time()
        print('Approx time: {:.0f} minutes'.format((current_time - start_time)*trainingsteps/60))

    if (step+1) % print_every == 0:
        current_time = time.time()
        print('step = {0}/{1}, {2:2.3f} s/step, time-to-go:{3:2.0f} min'.format(
                step+1, trainingsteps, (current_time - start_time) / (step + 1), 
            (current_time - start_time) / (step + 1) * (trainingsteps - step - 1)/60))

# Save model
if param.savemodel:
    torch.save(model.state_dict(), "savedmodels/" + model_name + ".pt")

    # TODO save details of the neural network
    print('Saved the model state_dict at savedmodels/' + model_name + '.pt' )

Approx time: 2 minutes


TypeError: box.rhs() missing 1 required positional argument: 'dev'

In [ ]:
if hybrid_method:
    np.save('hybrid_error_MC', Total_L2error)
    print('Completed hybrid method')
else:
    np.save('vanilla_error_MC', Total_L2error)
    print('Completed vanilla method')


Completed hybrid method


In [ ]:
X = torch.randn(10,1, device=dev)

In [ ]:
X.device

device(type='cuda', index=0)

In [ ]:
Y = X.clone()

In [ ]:
model.cpu()

FeedForwardNN(
  (sequential_model): Sequential(
    (0): Linear(in_features=2, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=20, bias=True)
    (2): Tanh()
    (3): Linear(in_features=20, out_features=20, bias=True)
    (4): Tanh()
    (5): Linear(in_features=20, out_features=20, bias=True)
    (6): Tanh()
    (7): Linear(in_features=20, out_features=1, bias=True)
  )
)